Notebook para criação de chaves primárias e secundárias

# Iniciando ambiente

# Import das bibliotecas

In [1]:
import findspark
from pyspark.sql import SparkSession
import sys
import os

## Incialização do spark

In [2]:
findspark.init()

## Criação da instância do Spark

In [3]:
spark =  SparkSession.builder\
            .master('local')\
            .appName('dados_enem')\
            .config("spark.executor.extraClassPath", os.path.abspath('./'))\
            .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/23 17:03:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Dados de conexão com bando de dados

In [4]:
url = f"jdbc:postgresql://localhost:5432/db_enem"
user = os.getenv('USER_DB')
password = os.getenv('PASSWORD_DB')